# Herramientas para Preprocesado de datos

## Importando las librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Importando los datos

In [ ]:
dataset = pd.read_csv('Data.csv')

In [ ]:
dataset.head(10)

## Separamos nuestros datos en vectores

In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

In [ ]:
print(y)

## Tratando los datos faltantes

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 1:3])
X[:, 1:3] = imputer.transform(X[:, 1:3])

In [ ]:
print(X)

## Codificando datos de categoría

### Codificando las Variables Independientes

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X_encoded = np.array(ct.fit_transform(X))

In [ ]:
print(X_encoded)

In [ ]:
X[:,0]

In [ ]:
cat_encoder = OneHotEncoder()
country_cat_encoder = cat_encoder.fit_transform(dataset[["Country"]])

In [ ]:
country_cat_encoder.toarray()

In [ ]:
cat_encoder.categories_

### Codificando la variable dependiente

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
print(y)

## Separando el dataset en datos de Entrenamiento y Prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size = 0.2, random_state = 1)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
print(y_train)

In [ ]:
print(y_test)

## Escalado de características

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
X_test[:, 3:] = sc.transform(X_test[:, 3:])

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
np.array(X_train[:,3:], dtype=np.float64).mean(axis=0)

In [ ]:
np.array(X_train[:,3:], dtype=np.float64).std(axis=0)

# ¿Escalado o Normalizado? ¿Cuál es la diferencia?

Una de las razones por las que es fácil confundirse entre el escalado y la normalización es porque los términos se utilizan a veces indistintamente y, para hacerlo aún más confuso, ¡son muy similares! En ambos casos, se transforman los valores de las variables numéricas para que los puntos de datos transformados tengan propiedades útiles específicas. La diferencia es que:

- en el **escalado**, se cambia el rango de los datos, mientras que
- en la **normalización**, se cambia la forma de la distribución de los datos.

## Escalado

Esto significa que está transformando sus datos para que se ajusten a una escala específica, como 0-100 o 0-1. Se desea escalar los datos cuando se utilizan métodos basados en medidas de la distancia entre los puntos de datos.

Por ejemplo, es posible que vea los precios de algunos productos tanto en yenes como en dólares estadounidenses. Un dólar estadounidense vale unos 100 yenes, pero si no se escalan los precios, métodos como SVM o KNN considerarán que una diferencia de precio de 1 yen es tan importante como una diferencia de 1 dólar estadounidense. Está claro que esto no encaja con nuestras intuiciones del mundo. Con la moneda, se puede convertir entre divisas. ¿Pero qué pasa si se trata de algo como la altura y el peso? No está del todo claro cuántas libras deben equivaler a una pulgada (o cuántos kilogramos deben equivaler a un metro).

Al escalar tus variables, puedes ayudar a comparar diferentes variables en igualdad de condiciones.

In [ ]:
np.random.seed(0)

original_data = np.random.exponential(size=1000).reshape(-1, 1)

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()

scaled_data = sc.fit_transform(original_data)

fig, ax = plt.subplots (1, 2, figsize=(15,3))
sns.histplot(original_data, ax=ax[0], kde=True, legend=False)
ax[0].set_title("Original Data")
sns.histplot(scaled_data, ax=ax[1], kde=True, legend=False)
ax[1].set_title("Scaled Data")
plt.show()

Observe que la forma de los datos **no cambia**, pero que en lugar de ir de 0 a 8, ahora va de 0 a 1.

## Normalización

La normalización es una transformación más radical. El objetivo de la normalización es cambiar las observaciones para que puedan describirse como una distribución normal.

[**Distribución normal**](https://en.wikipedia.org/wiki/Normal_distribution): También conocida como "curva de campana", se trata de una distribución estadística específica en la que un número aproximadamente igual de observaciones cae por encima y por debajo de la media, la media y la mediana son iguales, y hay más observaciones más cercanas a la media. La distribución normal también se conoce como distribución de Gauss.

En general, se normalizan los datos si se va a utilizar una técnica de machine learning o estadística que asume que los datos están distribuidos normalmente. Algunos ejemplos son el análisis discriminante lineal (LDA) y el método gaussiano de Naive-Bayes. ( Pro tip: cualquier método con "gaussiano" en el nombre probablemente asume la normalidad).

In [ ]:
from sklearn.preprocessing import normalize

from sklearn.preprocessing import power_transform


normalized_data = power_transform(original_data, method='box-cox')


fig, ax=plt.subplots(1, 2, figsize=(15, 3))
sns.histplot(original_data, ax=ax[0], kde=True, legend=False)
ax[0].set_title("Original Data")
sns.histplot(normalized_data, ax=ax[1], kde=True, legend=False)
ax[1].set_title("Normalized data")
plt.show()

La forma de nuestros datos ha cambiado. Antes de la normalización tenía casi forma de L. Pero después de la normalización se parece más al contorno de una campana (de ahí lo de "curva de campana")

## Pruebas de Normalidad
### Shapiro-Wilk Test

La prueba de Shapiro-Wilk evalúa una muestra de datos y cuantifica la probabilidad de que los datos se hayan extraído de una distribución gaussiana, llamada así por Samuel Shapiro y Martin Wilk.

En la práctica, se cree que la prueba de Shapiro-Wilk es una prueba fiable de normalidad, aunque hay algunos indicios de que la prueba puede ser adecuada para muestras de datos más pequeñas, por ejemplo, miles de observaciones o menos.

La función `shapiro()` de SciPy calculará la prueba de Shapiro-Wilk en un conjunto de datos determinado. La función devuelve tanto el estadístico W calculado por la prueba como el valor p.

La prueba supone que la muestra se ha extraído de una distribución gaussiana. Técnicamente, esto se denomina hipótesis nula, o H0. Se elige un nivel de umbral denominado alfa, normalmente el 5% (o 0,05), que se utiliza para interpretar el valor p.

- Si el valor p ≤ 0,05, rechazamos la hipótesis nula, es decir, asumimos que la distribución de nuestra variable no es normal/gaussiana.
- Si el valor p > 0,05, entonces no rechazamos la hipótesis nula, es decir, asumimos que la distribución de nuestra variable es normal/gaussiana.

In [ ]:
from scipy.stats import shapiro

print("Original Data")
stat, p = shapiro(original_data)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')
    
print("==============================")
print("Scaled Data")
stat, p = shapiro(scaled_data)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')
    
print("==============================")    
print("Normalized Data")
stat, p = shapiro(normalized_data)
print('Statistics=%.3f, p=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')